In [3]:
import tensorflow as tf
import scipy as sp
import numpy as np
import json

In [5]:
with open('time data and backups/id_pubs.json') as file:
    # id_pubs = {}
    # for pubs in user_pub.values():
    #     if pubs:
    #         for pub in pubs:
    #             if pub not in id_pubs.keys():
    #                 id_pubs[pub] = k
    #                 k += 1
    # json.dump(id_pubs,file)
    id_pubs = json.load(file)
    # print(len(id_pubs))
k = 0
with open('time data and backups/id_users.json') as file:
    # id_users = {}
    # for user in user_pub.keys():a
    #     id_users[user] = k
    #     k += 1
    # json.dump(id_users,file)
    id_users = json.load(file)
    # print(len(id_users))
shape = len(id_users.keys()),len(id_pubs.keys())

In [6]:
arr = sp.sparse.load_npz('time data and backups/matrix big.npz')

In [17]:
class DSSM(tf.Module):
    def __init__(self,components):
        super().__init__()

        self.components = components

        self.U = None
        self.I = None

        self.opt = tf.optimizers.Adam(learning_rate=0.1)

        self.fl_init = False
    @tf.function
    def loss_u(self,variables,y_true):
        y = variables@self.I
        loss = tf.sqrt(tf.losses.mean_squared_error(y_true,y))
        return loss

    @tf.function
    def loss_i(self,variables,y_true):
        y = self.U@variables
        loss = tf.sqrt(tf.losses.mean_squared_error(y_true,y))
        return loss

    def __call__(self, data=None,mean=True,*args, **kwargs):
        if data is None:
            return self.U@self.I
        if mean:
            return np.mean(data-self.U@self.I)
        return data-self.U@self.I

    def fit(self, data, iterations=4, parralel_U = 1, parralel_I=1,batch=1000):
        data_sparse = type(data) == sp.sparse._csr.csr_matrix
        for epoch in range(iterations):

            print('epoch: ',epoch)
            if not self.fl_init:
                self.U = tf.Variable(np.random.uniform(0,1,(data.shape[0],self.components)))
                self.I = tf.Variable(np.random.uniform(0,1,(self.components,data.shape[1])))
                self.fl_init = True
                # print(self().shape)
            if epoch % 2 == 0:
                for i in range(data.shape[0]//parralel_U):
                    # for k in range(data.shape[1]//batch):
                        ids = tf.random.uniform((1000,),0,data.shape[1],dtype=tf.int32)

                        variables = tf.Variable(self.U[(i)*parralel_U:(i+1)*parralel_U],name=f'VarBatch{parralel_U}')

                        if data_sparse:
                            y_true = tf.constant(data[i*parralel_U:(i+1)*parralel_U,ids].toarray())
                        else:
                            y_true = tf.constant(data[i*parralel_U:(i+1)*parralel_U,ids])

                        with tf.GradientTape(watch_accessed_variables=False) as tape:
                            tape.watch(variables)
                            loss = self.loss_u(variables,y_true)
                        # print('loss', loss)
                        gradients = tape.gradient(loss,variables)
                        # print('grads: ',gradients)
                        # if batch_U == 1:
                        self.opt.apply_gradients(zip([gradients],[variables]))
                        # else:
                        #     self.opt.apply_gradients(zip(gradients,[variables]))

                        self.U[(i)*parralel_U:(i+1)*parralel_U].assign(variables)

            else:
                for i in range(data.shape[0] // parralel_I):
                    # for k in range(data.shape[1]//batch):
                        ids = list(np.random.randint(0,data.shape[0],(batch,)))
                        variables = tf.Variable(self.I[(i) * parralel_I:(i + 1) * parralel_I], name=f'VarBatch{parralel_I}')
                        # print('v shape', variables.shape)
                        if data_sparse:
                            y_true = tf.constant(data[ids, i * parralel_I:(i + 1) * parralel_I].toarray())
                        else:
                            y_true = tf.constant(data[ids, i * parralel_I:(i + 1) * parralel_I])

                        with tf.GradientTape(watch_accessed_variables=False) as tape:
                            tape.watch(variables)
                            loss = self.loss_i(variables,y_true)
                        # print('loss', loss)
                        gradients = tape.gradient(loss,variables)
                        # print('grads: ',gradients)
                        # if batch_I == 1:
                        #     self.opt.apply_gradients(zip([gradients],[variables]))
                        # else:
                        #     self.opt.apply_gradients(zip(gradients,variables))
                        self.opt.apply_gradients(zip([gradients],[variables]))

                        self.I[:, (i) * parralel_I:(i + 1) * parralel_I].assign(variables)


In [13]:
data = arr[:25000]

In [18]:
model_big = DSSM(128)

In [19]:
model_big.fit(data,4,1,1)

epoch:  0


InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [4]:
li_t = tf.random.uniform((100,10),0,10000,dtype=tf.int32)

In [10]:
indeces = tf.random.uniform((10,),0,1000,dtype=tf.int32)

In [16]:
tf.gather(li_t,indeces,)

<tf.Tensor: shape=(10, 10), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])>

In [11]:
data[indeces]

<10x1117971 sparse matrix of type '<class 'numpy.float64'>'
	with 572 stored elements in Compressed Sparse Row format>